# Notebook 1: Processing FASTQ files using QIIME2
**Kyle Metcalfe**  </br></br>

**Welcome to Ge/Bi/ESE 246!** In this first session, we'll give a brief overview on the theory behind 16S rRNA amplicon sequencing and its use in modern microbial ecology, using some work I've done in my graduate studies as a tutorial. We'll then dive in and look at the raw data produced by the sequencing platform, and proceeed to process these data using [QIIME2](https://docs.qiime2.org/2019.10/).

**16S rRNA sequencing reveals microbial diversity**

Advances in DNA sequencing have transformed microbial ecology by providing access to the genetic information of microorganisms that have thus far evaded laboratory culture. Sequencing of one particular gene–that coding for the 16S subunit of ribosomal RNA–has been of particular utility for identifying the taxonomic composition of microbial communities. The 16S ribosomal RNA (rRNA) gene (and its eukaryotic equivalent, the 18S rRNA gene) is useful as it is present in virtually all cellular organisms, as it provides critical structural support to the ribosome and binds incoming mRNA. As a result of the important role the 16S rRNA gene plays in translation, its sequence is highly-conserved across taxa, preventing the sequence from becoming 'saturated' with mutations which would produce spurious results upon comparison. Thus, sequencing of the 16S rRNA gene has been used to directly compare cellular organisms [across the tree of life](https://www-pnas-org.clsproxy.library.caltech.edu/content/74/11/5088), giving us a tool to quickly place any given 16S rRNA sequence within a near-universal phylogenetic framework. 

**Next-generation sequencing revolutionizes microbial ecology**

Prior to the advent of high-throughput sequencing platforms, such as [Illumina's MiSeq platform](https://www.youtube.com/watch?v=fCd6B5HRaZ8), the enterprising microbial ecologist was limited to using techniques such as [Sanger sequencing](https://www-pnas-org.clsproxy.library.caltech.edu/content/74/12/5463), capable of recovering a single sequence ('read') for each sample provided. Thus, studies looking at 16S sequences recovered from uncultured prokaryotes found in the environment focused on the diversity these sequences exhibited. Typically, these studies PCR-amplified the whole 16S rRNA gene (~1500 base pairs) using primers designed to bind universally to Bacteria and Archaea 16S. Limited to ~10$^2$ sequences, each study could only evaluate the structure or composition of different communities semi-quantitatively.

In contrast, 'next-generation' sequencing platforms like MiSeq can provide to 16S rRNA sequencing studies many (10$^5$-10$^6$) reads per sample, albeit at somewhat more limited phylogenetic resolution, as PCR amplification is typically performed to produce a 'library' of ~400 bp fragments ('amplicons') of the 16S gene that are then sequenced. While a shorter sequence provides less data used to place a given sequence into the tree of life, it can be easily sequenced on a next-generation platform.  The resulting data products can then be analyzed for far more robust analysis of  community structure and composition within a given sample and across habitats, using tools such as Jari Oksanen's famous R package, [vegan](https://cran.r-project.org/web/packages/vegan/vegan.pdf), which we'll use often later in the course.

**The project**

Now that we have the historical context, let's jump into looking at real data. For this tutorial, we'll be looking at samples I collected as a part of my work applying [heavy-liquids separation](https://en.wikipedia.org/wiki/Sodium_metatungstate) to [methane seep](https://en.wikipedia.org/wiki/Cold_seep) sediments. The idea here was to explore the potential utility of this technique to purify microbial communities that associate with minerals such as [pyrite](https://www.mindat.org/min-3314.html) and [magnetite](https://www.mindat.org/min-2538.html) that are often found in marine sediments. Conductive materials, including magnetite, have been shown by [Amelia-Elena Rotaru and coworkers](https://mbio.asm.org/content/9/3/e00226-18.full) to stimulate direct, interspecies electron transfer (DIET) from acetate-oxidizing Bacteria to methanogenic Archaea. In methane seep sediments, [multicellular aggregates of archaeal methanotrophs (ANME) and sulfate-reducing Bacteria (SRB)](https://science.sciencemag.org/content/293/5529/484.long) [are also thought to use DIET](https://www.nature.com/articles/nature15512) to instead perform [anaerobic methane oxidation (AOM)](https://www.annualreviews.org/doi/full/10.1146/annurev.micro.61.080706.093130?url_ver=Z39.88-2003&rfr_id=ori%3Arid%3Acrossref.org&rfr_dat=cr_pub%3Dpubmed). Given that conductive minerals including magnetite have been shown to stimulate DIET and the importance of DIET in AOM syntrophy, we hypothesized that certain ANME and/or SRB taxa may preferentially associate with magnetite or pyrite surfaces. We further hypothesized that we could  detect this association in 16S amplicon libraries prepared from the bulk sediment and a from dense, pyrite- and magnetite-rich fraction of methane seep sediments. 

Here, we'll start the process of exploring this hypothesis by processing the raw data from a MiSeq platform operated by [Laragen, Inc.](http://www.laragen.com/laragen_nextgen.php). Prior to sequencing, PCR amplification targeted a [411 base pair region](https://sfamjournals.onlinelibrary.wiley.com/doi/full/10.1111/1462-2920.13023) of the 16S rRNA gene from replicate samples of methane seep sediments separated by density using sodium metatungstate. After paired-end sequencing on a MiSeq, amplicon libraries each appear in separate [CASAVA 1.8](http://illumina.bioinfo.ucr.edu/ht/documentation/data-analysis-docs/CASAVA-FASTQ.pdf/view) [FASTQ files](https://support.illumina.com/bulletins/2016/04/fastq-files-explained.html). Each FASTQ file consists of a number of reads (individual sequences detected by the sequencing platform) annotated by the quality of each base-pair call within each sequence. To start, make sure you've navigated to your directory at `/export/data1/projects/Ge246_2020`. Let's take a look at one of the FASTQ files (KM-052718-29-1_S88_L001_R1_001.fastq):

In [ ]:
nano ../KM-052718-29-1_S88_L001_R1_001.fastq

Press `control`+`X` to exit. Each entry here represents one sequence detected by the MiSeq platform. A header appears on the first line, describing the run identifier and the barcode sequences (`ATCTCAGG`,`GTAAGGTG`) that were attached to the PCR product before sequencing, identifying the sample from which this sequence was drawn. It is important to note here that our samples are already de-multiplexed, meaning that all sequences in each file will have the same barcode set. The sequence itself follows, followed by ASCII characters [describing the quality score](http://drive5.com/usearch/manual/quality_score.html).

Now that we have an understanding of the raw data we'll be processing, lets walk through how we'll take these FASTQ files and produce data we can then interpret to infer something about the microbial ecology of our samples. We'll start by loading QIIME2. Make sure you've navigated to a directory on the server using `cd` where the output will be printed to.

In [ ]:
module load anaconda3/2019.07
conda activate qiime2-2019.7

We'll then import the raw data into QIIME2, producing a data format that QIIME2 calls 'artifacts', with the extension `.qza`. This file format is useful to pass between QIIME2 functions.

In [ ]:
qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path ../Data \
  --input-format CasavaOneEightSingleLanePerSampleDirFmt \
  --output-path demux-paired-end.qza

Now that we have the data loaded, the next step is to identify sequences that by some metric are different taxa. Traditionally (i.e. in QIIME1), this has been done using a ['greedy'](https://drive5.com/usearch/manual/uclust_algo.html) algorithm to cluster sequences by similarity into 'Operational Taxonomic Units' (OTUs), at some % measure of their similarity. As you read through the linked description, what strikes you about the degree of reproducibility of this algorithm?

The algorithm central to QIIME2 sequence clustering is instead [DADA2](https://benjjneb.github.io/dada2/tutorial.html), which takes into account the additional data provided by modelling the rates of sequencing errors in a given dataset to give an estimate of the probability that the difference between two given sequences was not produced by sequencing error, the null hypothesis. The authors of DADA2 assert that this algorithm is sufficiently precise as to produce sequence clusters that represent sequences that more accurately represent the sequence mix in the PCR product, and thus call them 'Amplicon Sequence Variants' (AVSs). In the code blocks below, we produce a table of ASVs from the samples, using some options to handle the forward and reverse sequences I found to work well.

In [ ]:
qiime dada2 denoise-paired \
  --i-demultiplexed-seqs demux-paired-end.qza \
  --p-trunc-len-f 240 \
  --p-trunc-len-r 200 \
  --p-n-threads 0 \
  --o-table table.qza \
  --o-representative-sequences rep-seqs.qza \
  --o-denoising-stats denoising-stats.qza

In [ ]:
qiime metadata tabulate \
  --m-input-file denoising-stats.qza \
  --o-visualization denoising-stats.qzv

In [ ]:
qiime feature-table tabulate-seqs \
  --i-data rep-seqs.qza \
  --o-visualization rep-seqs.qzv

</br>For comparison, let's run the same `.qza` file through the an algorithm related to the old QIIME1 UCLUST default (UCLUST is only licensed for use in QIIME1) so we can compare the results:

In [ ]:
qiime vsearch join-pairs \
  --i-demultiplexed-seqs demux-paired-end.qza \
  --o-joined-sequences vsearch_joined.qza

In [ ]:
qiime vsearch dereplicate-sequences \
  --i-sequences vsearch_joined.qza \
  --o-dereplicated-table vsearch_table.qza \
  --o-dereplicated-sequences vsearch_rep-seqs.qza

In [ ]:
qiime vsearch cluster-features-de-novo \
  --i-table vsearch_table.qza \
  --i-sequences vsearch_rep-seqs.qza \
  --p-perc-identity 0.99 \
  --o-clustered-table vsearch_table-dn-99.qza \
  --o-clustered-sequences vsearch_rep-seqs-dn-99.qza

In [ ]:
qiime feature-table tabulate-seqs \
  --i-data vsearch_rep-seqs-dn-99.qza \
  --o-visualization vsearch_rep-seqs-dn-99.qzv

</br>Now that we have OTU and ASV tables, let's assign taxonomy to these OTUs and ASVs. We'll first use one of QIIME2's default classifiers, trained on Greengenes 13_8, a database of aligned 16S sequences. Here, we'll use 16S as a taxonomic framework, but it is important to note that there are [other frameworks](https://www.nature.com/articles/nbt.4229) that use a set of many conserved genes in addition to 16S to assign taxonomy.

In [ ]:
qiime feature-classifier classify-sklearn \
  --i-classifier /export/data1/db/16S_tag_processing_db/gg-13-8-99-515-806-nb-classifier.qza \
  --i-reads rep-seqs.qza \
  --o-classification taxonomy_ASVs_gg.qza

In [ ]:
qiime feature-classifier classify-sklearn \
  --i-classifier /export/data1/db/16S_tag_processing_db/gg-13-8-99-515-806-nb-classifier.qza \
  --i-reads vsearch_rep-seqs-dn-99.qza \
  --o-classification taxonomy_OTUs_gg.qza

Now, let's compare these results to the taxonomy assignments produced by the default Greengenes classifier to one trained on a modified [SILVA database](https://www.arb-silva.de/documentation/release-119/) we use in our lab, which consists of 16S sequences from clone libraries prepared from our seep sediment samples.

In [ ]:
qiime feature-classifier classify-sklearn \
  --i-classifier /export/data1/db/16S_tag_processing_db/SILVA_119_14_07_19_SAC_classifier.qza \
  --i-reads rep-seqs.qza \
  --o-classification taxonomy_ASVs_119.qza

In [ ]:
qiime feature-classifier classify-sklearn \
  --i-classifier /export/data1/db/16S_tag_processing_db/SILVA_119_14_07_19_SAC_classifier.qza \
  --i-reads vsearch_rep-seqs-dn-99.qza \
  --o-classification taxonomy_OTUss_119.qza

</br>Finally, let's produce simple barplot visualizations of our data. What do you notice in comparing this dataset when processed to produce OTUs vs. ASVs, and how do the barplots differ with different taxonomy assignment databases?

In [ ]:
qiime taxa barplot \
  --i-table table.qza \
  --i-taxonomy taxonomy_ASVs_132.qza \
  --m-metadata-file ../Notebook_1_Metadata.txt \
  --o-visualization taxa-bar-plots_ASVs_132.qzv

In [ ]:
qiime taxa barplot \
  --i-table vsearch_table-dn-99.qza \
  --i-taxonomy taxonomy_OTUs_132.qza \
  --m-metadata-file ../Notebook_1_Metadata.txt \
  --o-visualization taxa-bar-plots_OTUs_132.qzv

In [ ]:
qiime taxa barplot \
  --i-table table.qza \
  --i-taxonomy taxonomy_OTUs_119.qza \
  --m-metadata-file ../Notebook_1_Metadata.txt \
  --o-visualization taxa-bar-plots_ASVs_119.qzv

In [ ]:
qiime taxa barplot \
  --i-table vsearch_table-dn-99.qza \
  --i-taxonomy taxonomy_OTUs_132.qza \
  --m-metadata-file ../Notebook_1_Metadata.txt \
  --o-visualization taxa-bar-plots_OTUs_119.qzv

In [ ]:
qiime tools export \
  --input-path feature-table.qza \
  --output-path exported-feature-table